In [3]:
import os
import torch
from sklearn import metrics
import logging
from timeit import default_timer as timer
import pandas as pn

In [4]:
from context import resnet
from context import utils
from context import data_loader
from context import data_utils

In [4]:
# GENERATES CLASSIFICATION REPORT FOR 

BATCH_SIZE = 2
STATE_DICT_PATH = '../state'
DATA_PATH = '../data/imagenet_full'
STORE_RESULT_PATH = ''

In [4]:
def evaluateModel(model, loader, classes, batch_size): 
    pred, grndT = [], []
    for (images, labels) in iter(loader):
        if torch.cuda.is_available():
            images = images.cuda(non_blocking=True)
            labels = labels.cuda(non_blocking=True)
            
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        pred = pred + [classes[predicted[k]] for k in range(min(batch_size, labels.shape[0]))]
        grndT = grndT + [classes[labels[j]] for j in range(min(batch_size, labels.shape[0]))]
    return grndT, pred

def timed_decorator(function):
    def inner(*args, **kwargs):
        start = timer()
        result = function(*args, **kwargs)
        end = timer()
        print(f'{(end - start) * 1000} [ms]')
        return result
    return inner

@timed_decorator
def executeQualityBench(arch_name: str, loader, skip_n: int, classes, batch_size: int):
    model = utils.getModelWithOptimized(arch_name, n=skip_n, batch_size=batch_size)

    if torch.cuda.is_available():
        model = torch.nn.DataParallel(model).cuda()
    model.eval()
    arch = arch_name.split('-')[0]
    model, _, _, _ = utils.resumeFromPath(os.path.join(STATE_DICT_PATH, f'{arch}_model_best.pth.tar'), model)
    grndT, pred = evaluateModel(model, loader, classes, batch_size)

    #logging.debug(metrics.classification_report(grndT, pred, digits=3))
    #acc = metrics.accuracy_score(grndT, pred)
    #prec = metrics.precision_score(grndT, pred, average='macro')
    #rec = metrics.recall_score(grndT, pred, average='macro')
    #f1 = metrics.f1_score(grndT, pred, average='macro')
    return grndT, pred

In [5]:
experiment_data = list()

logging.basicConfig(level=logging.INFO)

model_name = 'resnet50-drop-rand-n'

skip_layers = 4
classes = data_utils.getLabelToClassMapping(DATA_PATH)
_, _, loader = data_loader.get_zipped_dataloaders(DATA_PATH, batch_size=BATCH_SIZE, use_valid=True)

../data/imagenet_red/index-val.txt
../data/imagenet_red/index-train.txt
../data/imagenet_red/index-val.txt


In [ ]:

name = model_name.split('-')[0]
print(f'Execute Non-Skip Run for {name}')
model = utils.getModel(name)
groundT, pred = executeQualityBench(name, loader, skip_n=0, classes=classes, batch_size=BATCH_SIZE)
experiment_data.append({
    'skip_n': 0,
    'ground_truth': groundT,
    'prediction': pred,
    'configuration': [],
    'acc': metrics.accuracy_score(groundT, pred)
})

for skip_layer in range(1, 7):
    for run in range(0, 30):
        print(f'Executing Run {run + 1} of 30')
        model = utils.getModelWithOptimized(model_name, n=skip_layers, batch_size=BATCH_SIZE)
        groundT, pred = executeQualityBench(model_name, loader, skip_n=skip_layers, classes=classes, batch_size=BATCH_SIZE)
        policy = resnet.DropPolicies.getDropPolicy()
        layer_config = []
        if isinstance(policy, resnet.DropPolicies.ResNetDropRandNPolicy):
            layer_config = policy.getSkipConfigurationList()
        experiment_data.append({
            'skip_n': skip_layers,
            'ground_truth': groundT, 
            'prediction': pred, 
            'configuration': layer_config,
            'acc': metrics.accuracy_score(groundT, pred)
            })

df = pn.DataFrame(experiment_data)
df.to_csv(os.path.join(STORE_RESULT_PATH, 'resnet50_experiment_results.csv'), index=False)

def print_data(data):
    for datum in data:
        print()
        print(datum['skip_n'])
        print(len(datum['ground_truth']))
        print(len(datum['prediction']))
        print(len(datum['configuration']))
        print()

In [6]:
temp = list()

temp.append({
    'skip_n': 1,
    'ground_truth': ['test2', 'test3', 'test3'], 
    'prediction': ['test1', 'test2', 'test3'], 
    'configuration': [True, False, False, True],
    'acc': 0.76
})
temp.append({
    'skip_n': 3,
    'ground_truth': ['test1', 'test1', 'test1'], 
    'prediction': ['test2', 'test2', 'test3'], 
    'configuration': [True, True, False, True],
    'acc': 0.70
})
temp.append({
    'skip_n': 1,
    'ground_truth': ['test3', 'test1', 'test3'], 
    'prediction': ['test1', 'test1', 'test3'], 
    'configuration': [False, False, False, False],
    'acc': 0.22
})


df1 = pn.DataFrame(temp)
df1

,skip_n,ground_truth,prediction,configuration,acc
0,1,"[test2, test3, test3]","[test1, test2, test3]","[True, False, False, True]",0.76
1,3,"[test1, test1, test1]","[test2, test2, test3]","[True, True, False, True]",0.70
2,1,"[test3, test1, test3]","[test1, test1, test3]","[False, False, False, False]",0.22


In [18]:
best = df[df['skip_n'] == 0].max()
print(metrics.classification_report(best.ground_truth, best.prediction, digits=3))

              precision    recall  f1-score   support

    no_plant      0.919     0.962     0.940       343
       plant      0.886     0.777     0.828       130

    accuracy                          0.911       473
   macro avg      0.903     0.870     0.884       473
weighted avg      0.910     0.911     0.909       473



3953
40
40
3953
40
40
3953
40
40
3953
40
40
3953
40
40


In [32]:
#######################################################################
# CODE TO CONVERT MULTI-CLASS CLASSIFICATION INTO BINARY CLASSFICIATION
# For DenseNet121
#######################################################################

for_skiped_layers = 6

BINARY_PATH = 'binary-classification/'

def convertStringToBoolArray(string: str):
    if type(string) == list:
        return string
    temp = [item.replace('[', '').replace(']', '').strip() for item in string.split(' ')]
    return [(x == 'True') for x in temp if len(x) == len('True') or len(x) == len('False')]

def convertStringToStringArray(string: str):
    return [
            x.replace('[', '')
            .replace(']', '')
            .replace("'", '')
            .strip() 
            for x in string.split(', ')
        ]

def filterForBinaryClassification(classes_list):
    plant_classes = [
        'Tulipa gesneriana', 'angiosperm', 
        'aster', 'floret', 'flower cluster', 
        'gazania', 'grape', 'hepatica', 
        'Night-blooming cereus', 'prairie gentian', 
        'raspberry', 'wild calla']
    temp_cls = []
    for name in classes_list:
        temp_cls.append('plant' if name in plant_classes else 'no_plant')
    return temp_cls

df = pn.read_csv(os.path.join('../reports', 'report-densenet121-run.csv'))

# conversion
df['ground_truth'] = df['ground_truth'].apply(lambda item: convertStringToStringArray(item))
df['prediction'] = df['prediction'].apply(lambda item: convertStringToStringArray(item))

# adaption for binary classification
df['ground_truth'] = df['ground_truth'].apply(lambda item: filterForBinaryClassification(item))
df['prediction'] = df['prediction'].apply(lambda item: filterForBinaryClassification(item))

entry = df[df['skip_n'] == for_skiped_layers]
grndT = entry.ground_truth.values[0]
pred = entry.prediction.values[0]

report_dict = metrics.classification_report(grndT, pred, digits=3, output_dict=True)
logging.info(metrics.classification_report(grndT, pred, digits=3))

temp_df = pn.DataFrame(report_dict).transpose()
file_path = os.path.join(BINARY_PATH, f'{for_skiped_layers}-densenet121-classification-report.csv')

if not os.path.exists(BINARY_PATH):
    os.mkdir(BINARY_PATH)
temp_df.to_csv(file_path, index=False)

